<a href="https://colab.research.google.com/github/siddtheshah/vc_modeling/blob/master/category_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install cityhash
import cityhash
import pandas

from copy import deepcopy

In [31]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [86]:
org_info = pd.read_csv("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/organizations.csv")
print(np.shape(org_info))
# Filter down the data purely to investees
org_info = org_info[org_info['primary_role'] == 'company']
print(np.shape(org_info))

(1089733, 41)
(1025256, 41)


In [88]:
org_hashes = org_info['uuid'].apply(cityhash.CityHash64)
org_info.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,legal_name,roles,domain,homepage_url,country_code,state_code,region,city,address,postal_code,status,short_description,category_list,category_groups_list,num_funding_rounds,total_funding_usd,total_funding,total_funding_currency_code,founded_on,last_funding_on,closed_on,employee_count,email,phone,facebook_url,linkedin_url,twitter_url,logo_url,alias1,alias2,alias3,primary_role,num_exits
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,organization,wetpaint,https://www.crunchbase.com/organization/wetpaint,123607.0,2007-05-25 13:51:27,2019-06-24 22:19:25,NaN,company,wetpaint.com,http://www.wetpaint.com/,USA,NY,New York,New York,902 Broadway 11th Floor New,10010,acquired,Wetpaint offers an online social publishing pl...,"Publishing,Social Media,Social Media Management","Content and Publishing,Internet Services,Media...",3.0,3.975000e+07,3.975000e+07,USD,2005-06-01,2008-05-19,NaN,51-100,info@wetpaint.com,206-859-6300,https://www.facebook.com/Wetpaint,https://www.linkedin.com/company/wetpaint,https://twitter.com/wetpainttv,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,NaN
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,zoho,https://www.crunchbase.com/organization/zoho,8777.0,2007-05-26 02:30:28,2018-10-27 00:29:49,NaN,"investor,company",zoho.com,https://www.zoho.com/,USA,CA,California,Pleasanton,4141 Hacienda Drive,94588,operating,"Zoho offers a suite of business, collaboration...","Cloud Computing,Collaboration,CRM,Developer To...","Information Technology,Internet Services,Priva...",NaN,NaN,NaN,NaN,1996-09-15,NaN,NaN,1001-5000,info@zohocorp.com,NaN,http://www.facebook.com/zoho,http://www.linkedin.com/company/zoho-corporati...,http://twitter.com/zoho,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,1.0
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,organization,digg,https://www.crunchbase.com/organization/digg,11537.0,2007-05-26 03:03:23,2018-12-10 10:09:14,"Digg Holdings, LLC",company,digg.com,http://www.digg.com,USA,NY,New York,New York,NaN,NaN,acquired,Digg Inc. operates a website that enables its ...,"Internet,Social Media,Social Network","Internet Services,Media and Entertainment",6.0,4.900000e+07,4.900000e+07,USD,2004-10-11,2016-09-13,NaN,51-100,feedback@digg.com,877-342-7222,http://www.facebook.com/digg,http://www.linkedin.com/company/digg,http://twitter.com/digg,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,NaN
4,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,organization,facebook,https://www.crunchbase.com/organization/facebook,15.0,2007-05-26 04:22:15,2020-03-30 18:33:35,"Facebook, Inc.","investor,company",facebook.com,http://www.facebook.com,USA,CA,California,Menlo Park,1 Hacker Way,94025,ipo,Facebook is an online social networking servic...,"E-Commerce,Mobile Apps,Social,Social Media,Soc...","Apps,Commerce and Shopping,Content and Publish...",15.0,2.335700e+09,2.335700e+09,USD,2004-02-04,2013-06-30,NaN,10000+,NaN,NaN,https://www.facebook.com/facebook/,http://www.linkedin.com/company/facebook,https://twitter.com/facebook,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,NaN
8,60485007-8856-bbac-aa1b-c535c41f5f47,Omnidrive,organization,omnidrive,https://www.crunchbase.com/organization/omnidrive,454088.0,2007-05-27 10:25:32,2019-10-03 06:30:53,NaN,company,omnidrive.com,http://www.omnidrive.com,USA,CA,California,Palo Alto,Suite 200 654 High Street,94301,closed,Omnidrive provides online storage facilities t...,"E-Commerce,File Sharing,Internet,Social Media,...","Commerce and Shopping,Internet Services,Media ...",1.0,8.000000e+05,8.000000e+05,USD,2005-11-01,2006-12-01,2008-09-15,unknown,info@omnidrive.com,NaN,http://www.facebook.com/Nomadesk,NaN,http://twitter.com/Nomadesk,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,NaN


In [33]:
import collections

# Select organization data
# Clean the df

comma_sep_list_vars = ['category_list', 'category_groups_list']
other_vars = ['uuid']
org_info_selected = org_info[other_vars + comma_sep_list_vars]
print("Rows to build vocabulary before NA drop: ", np.shape(org_info_selected)[0])
org_info_selected = org_info_selected.dropna()
org_info_selected = org_info_selected.reset_index(drop=True)

vocabulary = collections.Counter()

iter = 0
test_stop = -1
total_rows = np.shape(org_info_selected)[0]
print("Processing ", total_rows, " rows.")
for (i, row) in org_info_selected.iterrows():
  if iter == test_stop:
    break
  iter += 1
  if iter % 100000 == 0:
    print(iter, " of ", total_rows)

  for col, _ in row.iteritems():
    if col in comma_sep_list_vars:
      # Break categories into unigrams
      token_lists = [x.split(' ') for x in row[col].strip().split(',')]
      tokens = []
      for token_list in token_lists:
        for token in token_list:
          tokens.append(token)

      for token in tokens:
        vocabulary[token] += 1

# Show the unfiltered vocabulary counts
print(vocabulary.most_common())

# Filter low count words
low_count_threshold = 15
keys_to_remove = []
for word, count in vocabulary.items():
  if count < low_count_threshold:
    keys_to_remove.append(word)

for key in keys_to_remove:
  del vocabulary[key]

# Now rank and filter by document frequency. The top half frequent words will be dropped
key_list = list(vocabulary.keys())
fraction_to_drop = .25
chop = sorted(key_list, key=lambda x: vocabulary[x])[-int(fraction_to_drop*len(key_list)):]
for key in chop:
  del vocabulary[key]

# Create index to word mapping
word_to_index = {}
word_index = 1 # Leave space for adding the hash key.
for word, count in vocabulary.items():
  word_to_index[word] = word_index
  word_index += 1

print(word_to_index)
vocabulary_size = word_index

Rows to build vocabulary before NA drop:  1025256
Processing  963967  rows.
100000  of  963967
200000  of  963967
300000  of  963967
400000  of  963967
500000  of  963967
600000  of  963967
700000  of  963967
800000  of  963967
900000  of  963967
[('and', 783041), ('Software', 421264), ('Services', 376331), ('Information', 328481), ('Technology', 312525), ('Internet', 239214), ('Marketing', 190913), ('Health', 173889), ('Media', 169091), ('Care', 167984), ('Manufacturing', 167663), ('Entertainment', 134448), ('Advertising', 133623), ('Sales', 124540), ('Financial', 124035), ('Design', 122398), ('Hardware', 121259), ('Mobile', 119113), ('Shopping', 111194), ('Commerce', 104907), ('Real', 104381), ('Estate', 102213), ('Consumer', 98114), ('Apps', 98016), ('Engineering', 88802), ('Education', 86703), ('Science', 77933), ('Electronics', 77801), ('E-Commerce', 75006), ('Data', 73557), ('Analytics', 71562), ('Food', 69375), ('Consulting', 63881), ('Transportation', 61691), ('Energy', 59976),

In [34]:
# Show words from our filtered vocabulary.
print(vocabulary.most_common())

[('Location', 4184), ('Beauty', 4155), ('Project', 4081), ('Film', 4044), ('Aerospace', 4017), ('Wholesale', 3853), ('Diagnostics', 3828), ('Sharing', 3770), ('Outsourcing', 3768), ('Hospital', 3760), ('Planning', 3744), ('Life', 3715), ('Email', 3659), ('Mechanical', 3633), ('Rental', 3602), ('Crowdfunding', 3541), ('Cryptocurrency', 3510), ('Local', 3493), ('Therapeutics', 3475), ('Water', 3240), ('Art', 3218), ('Architecture', 3172), ('Wine', 3057), ('And', 3057), ('Spirits', 3057), ('Production', 2905), ('Distribution', 2798), ('Streaming', 2742), ('Advice', 2739), ('Collaboration', 2696), ('Investment', 2624), ('Employment', 2608), ('Packaging', 2587), ('Furniture', 2586), ('Semiconductor', 2568), ('Interior', 2544), ('Electrical', 2532), ('Smart', 2491), ('Database', 2418), ('Document', 2382), ('Risk', 2377), ('Precious', 2363), ('Metals', 2363), ('Augmented', 2363), ('Networking', 2358), ('Language', 2344), ('Support', 2338), ('Drones', 2264), ('Storage', 2229), ('Decor', 2227),

In [91]:
# Verify that org uuid is clean
org_info_selected['uuid'].iloc[:200]
print(np.shape(org_info_selected))

(963967, 3)


In [95]:
from scipy.sparse import coo_matrix
import collections

# Load our filtered vocabulary if we've saved it on disk.

row_indices = []
col_indices = []
data_values = []

used_hashes = set()
lookup_index = []
lookup_value = []

iter = 0
test_stop = -1
total_rows = np.shape(org_info_selected)[0]

print("Processing ", total_rows, " rows.")
for (i, row) in org_info_selected.iterrows():
  if iter == test_stop:
    break
  iter += 1
  if iter % 100000 == 0:
    print(iter, " of ", total_rows)

  uuid = row['uuid']
  hash = cityhash.CityHash64(uuid)

  if hash in used_hashes:  # Hash collision!
    print(hash)
    continue          # which shouldn't really happen
  
  # if not orgs.eq(hash).any():
  #   print("Bad hash: ", hash)
  #   break
  used_hashes.add(hash)

  lookup_value.append(uuid)
  lookup_index.append(hash)

  row_indices.append(i)
  col_indices.append(0)
  data_values.append(hash)
    
  for col, _ in row.iteritems():
    if col in comma_sep_list_vars:
      # Break categories into unigrams
      unigram_counts = collections.Counter()
      token_lists = [x.split(' ') for x in row[col].strip().split(',')]
      for token_list in token_lists:
        for token in token_list:
          unigram_counts[token] += 1
      
      for unigram, count in unigram_counts.items():
        if unigram in vocabulary:
          row_indices.append(i)
          col_indices.append(word_to_index[unigram])
          data_values.append(1)

data_arr = np.asarray(data_values, dtype=np.uint64)
unclustered = coo_matrix((data_arr, (row_indices, col_indices)), dtype=np.uint64)
print(unclustered.toarray().shape)

Processing  963967  rows.
100000  of  963967
200000  of  963967
300000  of  963967
400000  of  963967
500000  of  963967
600000  of  963967
700000  of  963967
800000  of  963967
900000  of  963967
(963967, 564)


In [104]:
import scipy.sparse
# print(unclustered)
# print(len(lookup_index))
# print(len(lookup_value))

## Generate company lookup
lookup = pd.DataFrame.from_dict({'hash':lookup_index, 'uuid':lookup_value})

lookup.to_csv('/lookup.csv')

# category_features = pd.DataFrame.sparse.from_spmatrix(joined)
# category_features.to_pickle('/category_features.npz')

scipy.sparse.save_npz('/category_features.npz', unclustered)

In [103]:
print(lookup)
print(org_hashes)
data_arr = np.asarray(data_values, dtype=np.uint64)
unclustered = coo_matrix((data_arr, (row_indices, col_indices)), dtype=np.uint64)
print(unclustered)

                        hash                                  uuid
0       13685534557686295101  e1393508-30ea-8a36-3f96-dd3226033abd
1         764015621929367586  bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7
2       10846552445983457719  5f2b40b8-d1b3-d323-d81a-b7a8e89553d0
3        5087506707876194815  df662812-7f97-0b43-9d3e-12f64f504fbb
4        9094535307341385563  60485007-8856-bbac-aa1b-c535c41f5f47
...                      ...                                   ...
963962   9744251496066876000  e95f3f39-2555-455f-8f8c-78d0eb573b36
963963  13650824445832434760  574414f1-d504-47fb-bb4c-0ccb1992a786
963964  15062640940509416191  e690cf82-eda4-4065-a42f-1119708aae45
963965   9929788675484222851  d206b3b0-c576-4557-aee5-1fb0e11b0afe
963966  12539503581336717831  235a1159-90a2-4208-9dfd-80e65be66c21

[963967 rows x 2 columns]
0          13685534557686295101
1            764015621929367586
2          10846552445983457719
4           5087506707876194815
8           9094535307341385563
         

# Validation

In [105]:
org_hashes = org_info['uuid'].apply(cityhash.CityHash64)
check_val = '1db7770c-31d4-c5f0-3d66-7354842aeb82'
hash_val = cityhash.CityHash64(check_val)
print("Hash Value: ", hash_val)
org_hashes_view = org_hashes[org_hashes == hash_val]
print(org_hashes_view)
print(org_hashes.eq(hash_val).any())

category_features_array = scipy.sparse.coo_matrix(unclustered, dtype=np.uint64)
category_features_df = pd.DataFrame.sparse.from_spmatrix(category_features_array)
print(category_features_df.iloc[:, 0])
print(np.shape(category_features_df))
print(category_features_df.iloc[:, 0].nunique())
print(np.count_nonzero(category_features_df.iloc[:, 0].isin(org_hashes)))
print(np.shape(lookup))
print(np.count_nonzero(category_features_df.iloc[:, 0].isin(lookup['hash'])))
print(np.count_nonzero(lookup['uuid'].isin(org_info['uuid'])))
print(np.count_nonzero(lookup['hash'].isin(org_hashes)))

Hash Value:  7568830722034072647
304    7568830722034072647
Name: uuid, dtype: uint64
True
0         13685534557686295101
1           764015621929367586
2         10846552445983457719
3          5087506707876194815
4          9094535307341385563
                  ...         
963962     9744251496066876000
963963    13650824445832434760
963964    15062640940509416191
963965     9929788675484222851
963966    12539503581336717831
Name: 0, Length: 963967, dtype: Sparse[uint64, 0]
(963967, 564)
963967
963967
(963967, 2)
963967
963967
963967


In [76]:
df = pd.DataFrame({'col1': [1, 2, 3], 'col2': [4, 2, 2]})

print(np.count_nonzero(df['col1'].isin(df['col2'])))
print(df['col2'].nunique())

1
2
